<a href="https://colab.research.google.com/github/karsarobert/ChatGPT2025/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu langchain_community langchain-huggingface langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

# Chat & Query your PDF files

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## The Game plan


<img src="https://dl.dropboxusercontent.com/s/gxij5593tyzrvsg/Screenshot%202023-04-26%20at%203.06.50%20PM.png" alt="vectorstore">


<img src="https://dl.dropboxusercontent.com/s/v1yfuem0i60bd88/Screenshot%202023-04-26%20at%203.52.12%20PM.png" alt="retreiver chain">


In [3]:
# Download the PDF Reid Hoffman book with GPT-4 from his free download link
!wget -q https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf

### Basic Chat PDF


In [4]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [5]:
# location of the pdf file/files.
doc_reader = PdfReader('/content/impromptu-rh.pdf')

In [6]:
doc_reader

In [7]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [8]:
len(raw_text)

371090

In [9]:
raw_text[:500]

'Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4Impromptu: AmplIfyIng our HumAnIty tHrougH AI  \nby Reid Hoffman with GPT-4\nISBNs:  979-8-9878319-1-5 Trade Paperback  \n979-8-9878319-2-2 Hardcover \n979-8-9878319-0-8 Ebook\nCopyright 2023 Dallepedia LLC\nPublished by Dallepedia LLC. All rights reserved. No portion \nof this work may be reproduced in any form, with the limited \nexception of brief quotations in editorial reviews, without express \nwritten permission from the pub'

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [10]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [11]:
len(texts)

466

In [12]:
texts[20]

'Because, really, an AI book? When things are moving so \nquickly? Even with a helpful AI on hand to speed the process, \nany such book would be obsolete before we started to write it—\nthat’s how fast the industry is moving.\nSo I hemmed and hawed for a bit. And then I thought of a frame \nthat pushed me into action.\nThis didn’t have to be a comprehensive “book” book so much as \na travelog, an informal exercise in exploration and discovery, \nme (with GPT-4) choosing one path among many. A snapshot \nmemorializing—in a subjective and decidedly not definitive \nway—the AI future we were about to experience.\nWhat would we see? What would impress us most? What would \nwe learn about ourselves in the process? Well aware of the brief \nhalf-life of this travelog’s relevance, I decided to press ahead.\nA month later, at the end of November 2022, OpenAI released \nChatGPT, a “conversational agent,” aka chatbot, a modified \nversion of GPT-3.5 that they had fine-tuned through a process'

In [13]:
texts[10]

'one. They’re just going to stare at you with that smug \nlook on their face and say, “You know this is a serious \nsafety hazard. You could have a fire, you could have a \nlawsuit, you could have a bad Yelp review. You better get \non that, pal.” And then they leave, and you’re left with a \ndark corner of your dining room, where no one wants to \nsit, where the food looks unappetizing, where the mood \nis gloomy. And you think to yourself, how many restau-\nrant inspectors does it take to change a lightbulb? None. \nThey don’t change lightbulbs. They just make you feel \nbad about yours.\nPretty good, right? Here’s another:\nReid: How many restaurant inspectors does it take to \nchange a lightbulb? Answer in the style of the philoso -\npher Ludwig Wittgenstein. \nGPT-4:  The question of how many restaurant inspectors \nit takes to change a lightbulb cannot be answered with \na simple number. In fact, the answer to this question lies \nmore in the realm of language than it does in the

## Making the embeddings

In [14]:
# Download embeddings from OpenAI
# embeddings = OpenAIEmbeddings()

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings

# Making the embeddings using HuggingFaceEmbeddings
# You can choose different models from the HuggingFace library
# Here we use 'sentence-transformers/all-MiniLM-L6-v2' as an example
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
#hf_embeddings = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-multilingual-base")

# You can now use 'hf_embeddings' instead of 'embeddings' for creating the vectorstore


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
docsearch = FAISS.from_texts(texts, hf_embeddings)

In [17]:
docsearch.embedding_function

HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
query = "how does GPT-4 change social media?"
docs = docsearch.similarity_search(query)

In [19]:
len(docs)

4

In [20]:
docs[0]

Document(id='c50a1921-d1d4-499d-8306-3452c2d8a380', metadata={}, page_content='be spread across multiple pages. Getting your answer will still \nchew up a lot of your time.\nIn addition, imagine when your goal is to get a concise overview \nof some broader interest or theme, like those I suggested at \nthe top of this chapter (researching supply chain management \nissues during COVID-19, or a travel itinerary that reflects spe -\ncific interests).\nThe takeaway: in your overall quest for authoritative infor-\nmation, GPT-4 helps you start somewhere much closer to the \nfinish line than if you didn’t have it as a resource. \nMore importantly, it possesses this capability because it is able \nto access and synthesize the web’s information in a significantly \ndifferent way from existing information resources like Wikipe-\ndia or traditional search engines. Essentially, GPT-4 arranges \nvast, unstructured arrays of human knowledge and expres -\nsion into a more connected and interoperable

## Plain QA Chain

In [21]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*nn1mc2rG9sGuqH16abWGGw.png">

The stuff method is really nice because it’s pretty simple. We just put all of it into one prompt and send that to the language model and get back one response.

In [54]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [55]:
# check the prompt
print(chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [24]:
query = "who are the authors of the book?"
docs = docsearch.similarity_search(query, k=4)
chain.run(input_documents=docs, question=query)

<ipython-input-24-4a65d163b8e8>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


' Donald Johanson and di Cesare are the authors of the book mentioned in this context.'

In [29]:
query = "who is the author of the book?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

" I don't know. The context does not mention the author of the book. "

In [30]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=20)
chain.run(input_documents=docs, question=query)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4814 tokens (4558 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

### QA Chain with mapreduce

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*oTxuYZnlyVE-SlP5ydrX_g.png">


In [38]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [39]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=10)
chain.run(input_documents=docs, question=query)

' The book is authored by Donald Johanson. \n'

In [46]:
chain = load_qa_chain(OpenAI(),
                      chain_type="map_rerank",
                      return_intermediate_steps=True
                      )

query = "who are openai?"
docs = docsearch.similarity_search(query,k=10)
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'intermediate_steps': [{'answer': ' OpenAI is an organization that was founded in 2015 and aims to give individuals versatile AI tools for their careers, professional development, and economic autonomy. ',
   'score': '100'},
  {'answer': ' OpenAI is a company that has created a tool called GPT-4, which can assist with processing information and generating action in various environments. It is expected to become essential for professionals and other workers in the near future, providing quick overviews and personalized search results. ',
   'score': '80'},
  {'answer': ' This document does not answer the question', 'score': '0'},
  {'answer': " OpenAI's founders", 'score': '80'},
  {'answer': ' OpenAI is a company that released a chatbot called ChatGPT, which was a modified version of GPT-3.5 that was fine-tuned through a process called Reinforcement Learning through Human Feedback (RLHF). They also received a $10 billion investment from Microsoft and had 100 million monthly active us

In [47]:
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

# LLM példány - Using ChatOpenAI for compatibility with map_rerank
llm = ChatOpenAI(temperature=0)

# Dokumentum keresés
query = "who are openai?"
docs = docsearch.similarity_search(query, k=10)

# QA lánc betöltése (map_rerank típussal)
# Pass the ChatOpenAI instance to load_qa_chain
chain = load_qa_chain(llm, chain_type="map_rerank", return_intermediate_steps=True)

# Futtatás
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'intermediate_steps': [{'answer': 'OpenAI is an organization that released tools like DALL-E 2 and ChatGPT in 2022, allowing users to access AI tools for various purposes. It was co-founded by one of the initial funders in 2015.',
   'score': '80'},
  {'answer': 'OpenAI is a company that has created a tool called GPT-4, which is designed to assist people in various environments and tasks by processing information and generating actions. It is predicted to become essential for professionals and workers in the near future.',
   'score': '90'},
  {'answer': 'This document does not answer the question', 'score': '0'},
  {'answer': "OpenAI's founders created the organization in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. They aim for AI to be a decentralized, personally empowering force rather than a top-down, totalizing one.",
   'score': '100'},
  {'answer': 'OpenAI is a company that released ChatGPT, a conversatio

In [48]:
results['output_text']

"OpenAI's founders created the organization in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. They aim for AI to be a decentralized, personally empowering force rather than a top-down, totalizing one."

In [49]:
results['intermediate_steps']

[{'answer': 'OpenAI is an organization that released tools like DALL-E 2 and ChatGPT in 2022, allowing users to access AI tools for various purposes. It was co-founded by one of the initial funders in 2015.',
  'score': '80'},
 {'answer': 'OpenAI is a company that has created a tool called GPT-4, which is designed to assist people in various environments and tasks by processing information and generating actions. It is predicted to become essential for professionals and workers in the near future.',
  'score': '90'},
 {'answer': 'This document does not answer the question', 'score': '0'},
 {'answer': "OpenAI's founders created the organization in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. They aim for AI to be a decentralized, personally empowering force rather than a top-down, totalizing one.",
  'score': '100'},
 {'answer': 'OpenAI is a company that released ChatGPT, a conversational agent or chatbot, which is

In [52]:
# check the prompt
print(chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
---------
Question: what type was the car?
Helpful Answer: a sports car or an suv
Score: 60

Example #3

Context

## RetrievalQA
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [56]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [57]:
rqa("What is OpenAI?")

{'query': 'What is OpenAI?',
 'result': ' OpenAI is an artificial intelligence research and deployment organization that aims to develop and make accessible powerful AI tools to improve and enhance various aspects of human life, such as work and personal decision-making.',
 'source_documents': [Document(id='6e428524-926a-4b32-851e-935f0afaa5b6', metadata={}, page_content='OpenAI had created a tool that became “intelligent” enough \nto productively assist you in many different environments, in \nmany different ways.\nMuch of what we do as modern people—at work and beyond—\nis to process information and generate action. GPT-4 will mas-\nsively speed your ability to do these things, and with greater \nbreadth and scope. Within a few years, this copilot will fall \nsomewhere between useful and essential to most profession -\nals and many other sorts of workers. Without GPT-4, they’ll 6Impromptu: Amplifying Our Humanity Through AI\nbe slower, less comprehensive, and working at a great dis-\

In [58]:
query = "What does gpt-4 mean for creativity?"
rqa(query)['result']

' GPT-4 means for creativity is that it has the potential to improve the creative practices of various types of artists and creators. It can be used as a brainstorming and production aid in tasks such as writing, designing, and problem-solving. It is versatile and can potentially amplify the abilities and productivity of both beginners and experts in various fields.'

In [59]:
query = "what have the last 20 years been like for American journalism?"
rqa(query)['result']

' The last 20 years have been mostly bad news for American journalism, with the rise of the internet and competition for ad dollars from non-news players causing traditional business models to be destroyed. This has led to newspaper closures and job losses for newsroom employees.'

In [60]:
query = "how can journalists use GPT-4??"
rqa(query)['result']

'\nJournalists can use GPT-4 in a variety of ways. One potential use is to generate summaries of top news stories or specific topics for their readers. They can also use GPT-4 to search for specific information or to create counter-arguments to published op-eds. However, given the potential for GPT-4 to produce incorrect or biased information, it is important for journalists to carefully review and verify the outputs of the tool. Additionally, GPT-4 can be used as a research assistant, providing quick and on-demand access to a broad base of knowledge. However, it is important for journalists to also cross-check information with human experts to ensure accuracy and depth. Overall, GPT-4 can greatly increase productivity for journalists, but it is important for them to use it carefully and critically.'

In [61]:
query = "How is GPT-4 different from other models?"
rqa(query)['result']

' GPT-4 is different from other models because it is able to specialize in different tasks such as generating 3D models, lighting effects, or audio editing based on text input or other modalities. It is also able to assist in various creative practices, such as writing, summarizing, and translating. Additionally, it is highly versatile and can assist users of all skill levels in a wide range of tasks.'

In [62]:
query = "What is beagle Bard?"
rqa(query)['result']

" I don't know."